# Text File Encryption and Decryption Using AES-128 ECB Algorithm

Make sure all required packages are installed by running `pip install -r requirements.txt` in the PYNQ image terminal.

<div style="text-align:center">
    <img src="pictures/pynq_z2_board-1.png" alt="Image Alt Text" />
</div>

<p style="text-align:center">Switch "SW1" and RGB LED "LD4" on PYNQ-Z2 Board</p>

## Encryption:

1. Ensure that switch "SW1" on the PYNQ board is off (visually, if off, RGB LED LD4 is also off). Wait until the widgets update to show "Encryption/Decryption: Encryption".
2. Upload a text file (.txt file) in UTF-8 format, enter the encryption key (in hexadecimal), and select your desired padding option.
3. Wait until two download options appear. Choose to download the output in Base64 format.

## Decryption:

1. Ensure that switch "SW1" on the PYNQ board is on (visually, if on, RGB LED LD4 will light up in blue). Wait until the widgets update to show "Encryption/Decryption: Decryption".
2. Upload a text file (.txt file) in Base64 format, enter the decryption key (in hexadecimal).
3. Wait until two download options appear. Choose to download the output in UTF-8 format.

<div style="border: 1px solid #007bff; background-color: #cce5ff; padding: 10px; border-radius: 5px; margin: 10px 0;">
  <strong>ℹ️ Important:</strong>
  <p>
    The default configuration is set to delete each text file in the "AES-128" directory on the next boot. 
    Make sure to download the relevant output files to your local PC!
  </p>
  <p>
    You can disable this service by typing in the terminal:
    <pre style="background-color: #eee; padding: 10px; border-radius: 5px;">
sudo systemctl stop delete_txt_files.service
sudo systemctl disable delete_txt_files.service
    </pre>
    To start the service again, type in the terminal:
    <pre style="background-color: #eee; padding: 10px; border-radius: 5px;">
sudo systemctl enable delete_txt_files.service
sudo systemctl start delete_txt_files.service
    </pre>
  </p>
</div>



In [ ]:
import ipywidgets as widgets
from IPython.display import display, FileLink
from pynq import allocate, Overlay
import numpy as np
import base64
import os
import time
import threading

# Load the overlay
ol = Overlay("/home/xilinx/jupyter_notebooks/AES-128/overlays/AES_ECB_128_design.bit")

# Setup the DMA channels
dma_send = ol.axi_dma_0.sendchannel
dma_recv = ol.axi_dma_0.recvchannel

# Access the GPIO
gpio_switches = ol.axi_gpio_0
switches_channel = gpio_switches.channel1

# Create a Label for key options based on switch state
key_options_label_description = widgets.Label(
    value='Encryption/Decryption:'
)
key_options_label = widgets.Label(
    value='Encryption'  # Initial value, will be updated
)

# Combine the description and value labels in an HBox
key_options_hbox = widgets.HBox([key_options_label_description, key_options_label])

# Create a password input box with a label for the key
password_input = widgets.Password(
    value='',
    placeholder='Enter key',
    description='Input Key:',
    disabled=False,
    layout=widgets.Layout(width='365px')
)

# Create a dropdown for Padding options
padding_options = widgets.Dropdown(
    options=['Zero', 'PKCS#7'],
    value='PKCS#7',
    description='Padding:',
    disabled=False
)

# Create a file upload widget
upload_widget = widgets.FileUpload(
    accept='.txt',  # Accept .txt files
    multiple=False  # Accept only one file
)

# Create a button to submit the file
submit_button = widgets.Button(
    description='Submit',
    button_style='success'
)

# Create an output widget to display results
output_widget = widgets.Output()

def add_padding(file_bytes, padding_type):
    """Add padding to files_bytes if needed"""
    padding_required = 16 - (len(file_bytes) % 16) if len(file_bytes) % 16 != 0 else 0
    if padding_type == 'Zero':
        file_bytes += b'\x00' * padding_required
    elif padding_type == 'PKCS#7':
        file_bytes += bytes([padding_required] * padding_required)  
    return file_bytes

def save_buffer_txt(buffer_data, filename):
    """Save buffer data to text file."""
    with open(filename, 'w') as f:
        byte_list = ' '.join([format(byte, '02x') for byte in buffer_data])
        f.write(byte_list)

def save_buffer_base64(buffer_data, filename):
    """Save buffer data in base64 format to text file."""
    with open(filename, 'w') as f:
        base64_data = base64.b64encode(buffer_data).decode('utf-8')
        f.write(base64_data)

def save_buffer_utf8(buffer_data, filename):
    """Save buffer data in UTF-8 format to text file."""
    with open(filename, 'w') as f:
        utf8_data = buffer_data.tobytes().decode('utf-8', errors='replace')
        f.write(utf8_data)
        
def adjust_buffer_length(buffer, alignment=16):
    """Adjusts buffer length to be a multiple of alignment bytes."""
    current_length = len(buffer)
    new_length = (current_length + alignment - 1) // alignment * alignment  # Round up to the nearest multiple of alignment
    if new_length > current_length:
        buffer = np.concatenate((buffer, np.zeros(new_length - current_length, dtype=np.uint8)))
    return buffer, new_length

def remove_padding(buffer, padding_type):
    if padding_type == 'Zero':
        # Remove zero padding by trimming trailing zero bytes
        while buffer[-1] == 0x00:
            buffer = buffer[:-1]
        return buffer
    elif padding_type == 'PKCS#7':
        # Remove PKCS#7 padding
        padding_value = buffer[-1]
        # If buffer length is a multiple of 16, assume no padding was added
        if padding_value > 16:
            return buffer
        else: 
            while buffer[-1] == padding_value:
                buffer = buffer[:-1]
            return buffer   

def on_submit(b):
    errors = []
    
    # Validate input key
    key = password_input.value
    if not key:
        errors.append("Error: Key is required.")
    elif len(key) != 32 or not all(c in '0123456789abcdefABCDEF' for c in key):
        errors.append("Error: Key must be 16 bytes in hex format (32 characters).")
    
    # Validate file upload
    if not upload_widget.value:
        errors.append("Error: No file uploaded.")
    
    # Display errors or process the input
    with output_widget:
        output_widget.clear_output()
        if errors:
            for error in errors:
                print(error)
        else:
            uploaded_file = next(iter(upload_widget.value.values()))
            file_content = uploaded_file['content'].decode('utf-8')
            file_name = uploaded_file.get('metadata', {}).get('name', 'uploaded_file')
            
            if key_options_label.value == 'Encryption':
                # Ensure the text is encoded in UTF-8 with BOM
                file_bytes = file_content.encode('utf-8-sig')
                
                # Add padding
                file_bytes = add_padding(file_bytes, padding_options.value)
                
                total_size_in_bytes = 16 + len(file_bytes)
                
                # Allocate memory for the input buffer
                input_buffer = allocate(shape=(total_size_in_bytes,), dtype=np.uint8)
                
                # Convert the key to bytes and fill the buffer
                key_bytes = bytes.fromhex(key)
                input_buffer[:16] = np.frombuffer(key_bytes, dtype=np.uint8)
                input_buffer[16:] = np.frombuffer(file_bytes, dtype=np.uint8)
                
                # Calculate number of chunks
                chunk_size = 0x4000  # 16,384 bytes
                num_chunks = (total_size_in_bytes + chunk_size - 1) // chunk_size
                
                # Create final output buffer
                output_buffer_final = allocate(shape=(total_size_in_bytes - 16,), dtype=np.uint8)
                
                for i in range(num_chunks):
                    # Calculate start and end indices for the current chunk
                    start_idx = i * chunk_size
                    end_idx = min(start_idx + chunk_size, total_size_in_bytes - 16)
                    
                    # Prepare input buffer for the current chunk
                    chunk_data = file_bytes[start_idx:end_idx]
                    input_buffer = allocate(shape=(len(chunk_data) + len(key_bytes),), dtype=np.uint8)
            
                    # Add key to the input buffer
                    input_buffer[:len(key_bytes)] = np.frombuffer(key_bytes, dtype=np.uint8)
            
                    # Add chunk data to the input buffer
                    input_buffer[len(key_bytes):] = np.frombuffer(chunk_data, dtype=np.uint8)
            
                    # Adjust input buffer length to be a multiple of 16 bytes
                    input_buffer, new_length = adjust_buffer_length(input_buffer)
            
                   # Check if the chunk has data
                    if new_length > 16:
                        # Allocate output buffer for the current chunk
                        output_buffer = allocate(shape=(new_length - len(key_bytes),), dtype=np.uint8)
                
                        # Perform DMA transfer for the current chunk
                        dma_send.transfer(input_buffer)
                        dma_recv.transfer(output_buffer)
                        dma_send.wait()
                        dma_recv.wait()
                
                        # Copy the output buffer to the final output buffer
                        output_buffer_final[start_idx:end_idx] = output_buffer[:end_idx - start_idx]
                        
                        # Free the input and output buffers for the current chunk
                        del input_buffer
                        del output_buffer
                    else:
                        # Free the input buffer 
                        del input_buffer
                
                # Save output buffer data to text file (hex format)
                output_file_name_hex = f"encrypted_{file_name}_hex_ECB.txt"
                save_buffer_txt(output_buffer_final, output_file_name_hex)
                
                # Save output buffer data to text file (base64 format)
                output_file_name_base64 = f"encrypted_{file_name}_base64_ECB.txt"
                save_buffer_base64(output_buffer_final, output_file_name_base64)
                
                # Display download links for output buffer text files
                display(FileLink(output_file_name_hex, result_html_prefix="Download Output Buffer Hex Data: "))
                display(FileLink(output_file_name_base64, result_html_prefix="Download Output Buffer Base64 Data: "))
                
                # Free the final output buffer
                del output_buffer_final
                
            elif key_options_label.value == 'Decryption':
                try:
                    file_bytes = base64.b64decode(file_content)
                except Exception as e:
                    print(f"Error decoding base64 file: {e}")
                    return
                
                total_size_in_bytes = 16 + len(file_bytes)
                
                # Allocate memory for the input buffer
                input_buffer = allocate(shape=(total_size_in_bytes,), dtype=np.uint8)
                
                # Convert the key to bytes and fill the buffer
                key_bytes = bytes.fromhex(key)
                input_buffer[:16] = np.frombuffer(key_bytes, dtype=np.uint8)
                input_buffer[16:] = np.frombuffer(file_bytes, dtype=np.uint8)
                
                # Calculate number of chunks
                chunk_size = 0x4000  # 16,384 bytes
                num_chunks = (total_size_in_bytes + chunk_size - 1) // chunk_size
                
                # Create final output buffer
                output_buffer_final = allocate(shape=(total_size_in_bytes - 16,), dtype=np.uint8)
                
                for i in range(num_chunks):
                    # Calculate start and end indices for the current chunk
                    start_idx = i * chunk_size
                    end_idx = min(start_idx + chunk_size, total_size_in_bytes - 16)
                    
                    # Prepare input buffer for the current chunk
                    chunk_data = file_bytes[start_idx:end_idx]
                    input_buffer = allocate(shape=(len(chunk_data) + len(key_bytes),), dtype=np.uint8)
            
                    # Add key to the input buffer
                    input_buffer[:len(key_bytes)] = np.frombuffer(key_bytes, dtype=np.uint8)
            
                    # Add chunk data to the input buffer
                    input_buffer[len(key_bytes):] = np.frombuffer(chunk_data, dtype=np.uint8)
            
                    # Adjust input buffer length to be a multiple of 16 bytes
                    input_buffer, new_length = adjust_buffer_length(input_buffer)
            
                    # Check if the chunk has data
                    if new_length > 16:
                        # Allocate output buffer for the current chunk
                        output_buffer = allocate(shape=(new_length - len(key_bytes),), dtype=np.uint8)
                
                        # Perform DMA transfer for the current chunk
                        dma_send.transfer(input_buffer)
                        dma_recv.transfer(output_buffer)
                        dma_send.wait()
                        dma_recv.wait()
                
                        # Copy the output buffer to the final output buffer
                        output_buffer_final[start_idx:end_idx] = output_buffer[:end_idx - start_idx]
                        
                        # Free the input and output buffers for the current chunk
                        del input_buffer
                        del output_buffer
                    else:
                        # Free the input buffer 
                        del input_buffer
                    
                # Remove padding
                output_buffer_final = remove_padding(output_buffer_final, padding_options.value)
                
                # Save output buffer data to text file (hex format)
                output_file_name_hex = f"decrypted_{file_name}_base64_ECB.txt"
                save_buffer_txt(output_buffer_final, output_file_name_hex)
                
                # Save output buffer data to text file (UTF-8 format)
                output_file_name_utf8 = f"decrypted_{file_name}_UTF-8_ECB.txt"
                save_buffer_utf8(output_buffer_final, output_file_name_utf8)
                
                # Display download links for output buffer text files
                display(FileLink(output_file_name_hex, result_html_prefix="Download Decrypted Output Buffer Hex Data: "))
                display(FileLink(output_file_name_utf8, result_html_prefix="Download Decrypted Output Buffer UTF-8 Data: "))
                
                # Free the final output buffer
                del output_buffer_final

# Set up the button click event
submit_button.on_click(on_submit)

# Function to update the Label based on the switch state
def update_key_options_label():
    while True:
        switch_status = switches_channel.read()
        first_switch_status = switch_status & 0x01

        if first_switch_status:
            key_options_label.value = 'Decryption'
        else:
            key_options_label.value = 'Encryption'

        time.sleep(1)

# Start the switch monitoring in a separate thread
switch_monitoring_thread = threading.Thread(target=update_key_options_label)
switch_monitoring_thread.daemon = True
switch_monitoring_thread.start()

# Arrange the widgets vertically
widgets_layout = widgets.VBox([
    key_options_hbox,
    widgets.HBox([password_input]), 
    padding_options,
    upload_widget, 
    submit_button, 
    output_widget
])

# Display the arranged widgets
display(widgets_layout)